In [3]:
from behavior_and_time_alignment import beh_and_time_alignment
from session_preprocessing import ephys_opto_preprocessing, session_crosscorr, ephys_opto_crosscorr
from opto_tagging import opto_plotting_session, opto_plotting_unit, opto_tagged_spike_stability
from opto_waveforms_preprocessing import opto_wf_preprocessing
from drift_analysis import plot_session_opto_drift
import matplotlib.pyplot as plt
from utils.beh_functions import session_dirs
import numpy as np
import pandas as pd
import os
from utils.plot_utils import combine_pdf_big
import spikeinterface as si
from joblib import Parallel, delayed



In [4]:
session_assets = pd.read_csv('/root/capsule/code/data_management/session_assets.csv')
session_list = session_assets['session_id']
ani_list = []

In [2]:
# combine opto pdfs
data_type = 'curated'
def process(session, data_type):
    pdf_dir = session_dirs(session)[f'opto_dir_fig_{data_type}']
    output_pdf = os.path.join(session_dirs(session)[f'opto_dir_{data_type}'], f'{session}_opto_tagging_png.pdf')
    if os.path.exists(pdf_dir) and not os.path.exists(output_pdf):
        print(session)
        combine_pdf_big(pdf_dir, output_pdf)
# Parallel(n_jobs=4)(delayed(process)(session, data_type) for session in session_list)
for session in session_list:
    try:
        process(session, data_type)
        plt.close('all')
    except:
        print(f'Failed to process {session}')

NameError: name 'session_list' is not defined

In [ ]:
process('behavior_751004_2024-12-19_11-50-37', data_type) 

behavior_751004_2024-12-19_11-50-37
Combined PDF saved as: /root/capsule/scratch/751004/behavior_751004_2024-12-19_11-50-37/ephys/opto/curated/behavior_751004_2024-12-19_11-50-37_opto_tagging_png.pdf


In [ ]:
from joblib import Parallel, delayed
session_list = ['behavior_717121_2024-06-15_10-00-58',
                'behavior_716325_2024-05-31_10-31-14']
data_type = 'curated'
def process(session, data_type='raw'):
    plt.close('all')
    print(f'{session} start')
    beh_and_time_alignment(session) # plot behavior, check alignment between behavior, sound, and ephys
    plt.close('all')
    ephys_opto_preprocessing(session, data_type, 'soma') 
    # redo spiketimes if not aligned, make opto_table (laser time in nidaq); 
    # generate a pkl file with opto_resp/lat to each condition
    plt.close('all')
    ephys_opto_crosscorr(session, data_type)
    # separate session into opto/beh, cal cross-correlation
    plt.close('all')
    opto_wf_preprocessing(session, data_type, 'soma', load_sorting_analyzer=True)
    # recal wf based on conditions. (pre/post, power), cal similarity, euclidean distance, correlation with spont
    # pay attention to difference early and late in the session
    
    opto_tagging_df = opto_plotting_session(session, data_type, 'soma', plot=True, resp_thresh=0.4, lat_thresh=0.02)
    
    # save as pickle
    with open(os.path.join(session_dirs(session)[f'opto_dir_{data_type}'], f'{session}_opto_tagging_metrics.pkl'), 'wb') as f:
        opto_tagging_df.to_pickle(f)
    
    if '751004' in session:
        plot_session_opto_drift(session, data_type, update_csv=False)
    else:
        plot_session_opto_drift(session, data_type, update_csv=True)
    print(f'{session} stop')
        

# Parallel(n_jobs=2)(delayed(process)(session, data_type = data_type) for session in session_list[2:])


In [6]:
datatype = 'curated'
list_exclude = ['716325', '717121', '717259', '717263', '714116', '724172', '722832', '739977', '744779', '739970', '751181', '751766_2025-02-15_12-08-11']
def process(session, data_type):
    if not any(exclude in session for exclude in list_exclude):
        print(f'{session} start')
        try:
            plot_session_opto_drift(session, data_type, update_csv=True, plot=True)
        except:
            plot(f'{session} failed')   
Parallel(n_jobs=4)(delayed(process)(session, data_type = datatype) for session in session_list)

behavior_751004_2024-12-19_11-50-37 start
behavior_751004_2024-12-21_13-28-28 start
behavior_751004_2024-12-23_14-20-03 start
behavior_751769_2025-01-17_11-37-39 start


/opt/conda/lib/python3.10/site-packages/aind_dynamic_foraging_basic_analysis/plot/plot_foraging_session.py:504: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()
/opt/conda/lib/python3.10/site-packages/aind_dynamic_foraging_basic_analysis/plot/plot_foraging_session.py:504: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()
/opt/conda/lib/python3.10/site-packages/aind_dynamic_foraging_basic_analysis/plot/plot_foraging_session.py:504: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()
/opt/conda/lib/python3.10/site-packages/aind_dynamic_foraging_basic_analysis/plot/plot_foraging_session.py:504: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to

Merging file 0 out of 3
PDF files in '/root/capsule/scratch/751004/behavior_751004_2024-12-21_13-28-28/ephys/opto/curated/drift' merged into '/root/capsule/scratch/751004/behavior_751004_2024-12-21_13-28-28/ephys/opto/curated/behavior_751004_2024-12-21_13-28-28_drift.pdf' successfully.
behavior_751004_2024-12-22_13-09-17 start


/root/capsule/code/beh_ephys_analysis/drift_analysis.py:422: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()
/root/capsule/code/beh_ephys_analysis/drift_analysis.py:422: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()
/root/capsule/code/beh_ephys_analysis/drift_analysis.py:422: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()
/opt/conda/lib/python3.10/site-packages/aind_dynamic_foraging_basic_analysis/plot/plot_foraging_session.py:504: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout(

Merging file 0 out of 3
PDF files in '/root/capsule/scratch/751004/behavior_751004_2024-12-23_14-20-03/ephys/opto/curated/drift' merged into '/root/capsule/scratch/751004/behavior_751004_2024-12-23_14-20-03/ephys/opto/curated/behavior_751004_2024-12-23_14-20-03_drift.pdf' successfully.
behavior_751769_2025-01-16_11-32-05 start


...

/root/capsule/code/beh_ephys_analysis/drift_analysis.py:422: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()
/root/capsule/code/beh_ephys_analysis/drift_analysis.py:422: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()
/opt/conda/lib/python3.10/site-packages/aind_dynamic_foraging_basic_analysis/plot/plot_foraging_session.py:504: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()
/root/capsule/code/beh_ephys_analysis/drift_analysis.py:422: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend(

Merging file 0 out of 9
PDF files in '/root/capsule/scratch/751766/behavior_751766_2025-02-14_11-37-11/ephys/opto/curated/drift' merged into '/root/capsule/scratch/751766/behavior_751766_2025-02-14_11-37-11/ephys/opto/curated/behavior_751766_2025-02-14_11-37-11_drift.pdf' successfully.


/root/capsule/code/beh_ephys_analysis/drift_analysis.py:422: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()
/root/capsule/code/beh_ephys_analysis/drift_analysis.py:422: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()
/root/capsule/code/beh_ephys_analysis/drift_analysis.py:422: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()


Merging file 0 out of 3
PDF files in '/root/capsule/scratch/751766/behavior_751766_2025-02-13_11-31-21/ephys/opto/curated/drift' merged into '/root/capsule/scratch/751766/behavior_751766_2025-02-13_11-31-21/ephys/opto/curated/behavior_751766_2025-02-13_11-31-21_drift.pdf' successfully.


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [ ]:
for session in session_list:
    try:
        process(session, data_type)
    except:
        print(f'Failed to process {session}')

behavior_717121_2024-06-15_10-00-58 start
behavior_717121_2024-06-15_10-00-58


/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:7335: RuntimeWarning: invalid value encountered in divide
  terms = (f_obs - f_exp)**2 / f_exp


behavior_717121_2024-06-15_10-00-58


/opt/conda/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.6.0-alpha because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/conda/lib/python3.10/site-packages/spikeinterface/core/base.py:1109: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.7 is recommended
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter return_scaled from spike_amplitudes extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.10/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter metrics_kwargs from template_metrics extension
  warnings.warn(f"Removing legacy parameter {p} 

48 out of 243 units pass quality control
{'samples_to_keep': [-30, 60], 'y_neighbors_to_keep': 3, 'orginal_loc': False}


...

/opt/conda/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:2999: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/conda/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3000: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/opt/conda/lib/python3.10/site-packages/spikeinterface/core/base.py:1109: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.7 is recommended
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter return_scaled from spike_amplitudes extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.10/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter metrics_kwargs from template_metrics extension
  wa

76 out of 281 units pass quality control
{'samples_to_keep': [-30, 60], 'y_neighbors_to_keep': 3, 'orginal_loc': False}


/root/capsule/code/beh_ephys_analysis/opto_tagging.py:390: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  opto_wf = opto_wf.apply(pd.to_numeric, errors='ignore')


Merging file 0 out of 281
Merging file 50 out of 281
Merging file 100 out of 281
Merging file 150 out of 281
Merging file 200 out of 281
Merging file 250 out of 281
PDF files in '/root/capsule/scratch/716325/behavior_716325_2024-05-31_10-31-14/ephys/opto/curated/figures' merged into '/root/capsule/scratch/716325/behavior_716325_2024-05-31_10-31-14/ephys/opto/curated/behavior_716325_2024-05-31_10-31-14_opto_tagging.pdf' successfully.
15 out of 281 units pass quality control and opto tagging


Failed to process behavior_716325_2024-05-31_10-31-14
behavior_716325_2024-05-31_10-31-14
Combined PDF saved as: /root/capsule/scratch/716325/behavior_716325_2024-05-31_10-31-14/ephys/opto/curated/behavior_716325_2024-05-31_10-31-14_opto_tagging_png.pdf
There are multiple recordings in the curated nwb directory. Please specify the recording you would like to use.
Merging file 0 out of 8
Merging file 0 out of 3
PDF files in '/root/capsule/scratch/751004/behavior_751004_2024-12-19_11-50-37/ephys/opto/curated/drift' merged into '/root/capsule/scratch/751004/behavior_751004_2024-12-19_11-50-37/ephys/opto/curated/behavior_751004_2024-12-19_11-50-37_drift.pdf' successfully.
PDF files in '/root/capsule/scratch/751004/behavior_751004_2024-12-23_14-20-03/ephys/opto/curated/drift' merged into '/root/capsule/scratch/751004/behavior_751004_2024-12-23_14-20-03/ephys/opto/curated/behavior_751004_2024-12-23_14-20-03_drift.pdf' successfully.
Merging file 0 out of 5
PDF files in '/root/capsule/scratch/

In [ ]:
opto_tagging_df = opto_plotting_session('behavior_751004_2024-12-19_11-50-37', 'curated', 'soma', plot=False, resp_thresh=0.4, lat_thresh=0.02)

/opt/conda/lib/python3.10/site-packages/spikeinterface/core/base.py:1109: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.101.2 is recommended
  warnings.warn(
/root/capsule/code/beh_ephys_analysis/opto_tagging.py:351: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  unit_qc = unit_qc.apply(pd.to_numeric, errors='ignore')


91 out of 267 units pass quality control
{'samples_to_keep': [-30, 60], 'y_neighbors_to_keep': 3, 'orginal_loc': False}


/root/capsule/code/beh_ephys_analysis/opto_tagging.py:390: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  opto_wf = opto_wf.apply(pd.to_numeric, errors='ignore')


Merging file 0 out of 267
Merging file 50 out of 267
Merging file 100 out of 267
Merging file 150 out of 267
Merging file 200 out of 267
Merging file 250 out of 267
PDF files in '/root/capsule/scratch/751004/behavior_751004_2024-12-19_11-50-37/ephys/opto/curated/figures' merged into '/root/capsule/scratch/751004/behavior_751004_2024-12-19_11-50-37/ephys/opto/curated/behavior_751004_2024-12-19_11-50-37_opto_tagging.pdf' successfully.
4 out of 267 units pass quality control and opto tagging


In [ ]:
# input: sessio_id
# output: data_frame, columns: unit_id, anti_sti metrics, plots

In [ ]:
opto_tagging_df

,unit_id,resp_p,resp_lat,powers,sites,num_pulses,durations,freqs,stim_times,opto_pass,...,ks_unit_id,isi_violations_ratio,firing_rate,presence_ratio,amplitude_cutoff,decoder_label,depth,euclidean_norm,correlation,default_qc
0,0,None,None,None,None,None,None,None,None,False,...,0,0.000000,0.072709,0.808511,NaN,mua,170.43,NaN,NaN,False
1,1,None,None,None,None,None,None,None,None,False,...,1,0.486594,0.918804,0.872340,0.000383,mua,3.40,NaN,NaN,False
2,2,[0.72],[0.011866555440285469],[50],[surface_LC],[5],[5],[5],[post],True,...,2,0.048411,4.799117,1.000000,0.000168,sua,78.83,[0.011572034337162935],[0.9979395187981891],True
3,3,"[0.9, 0.9, 0.7, 0.8]","[0.0108739817788934, 0.01263726076493705, 0.01...","[10, 20, 30, 50]","[surface_LC, surface_LC, surface_LC, surface_LC]","[5, 5, 5, 5]","[4, 4, 4, 4]","[5, 5, 5, 5]","[post, post, post, post]",True,...,3,0.074913,10.321623,1.000000,0.000033,sua,94.24,"[0.019803436709564345, 0.020649075605866287, 0...","[0.9864396155122652, 0.9864571959817784, 0.983...",True
4,5,None,None,None,None,None,None,None,None,False,...,5,0.000326,60.011850,1.000000,0.000288,sua,160.25,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,295,None,None,None,None,None,None,None,None,False,...,295,0.000000,0.032921,0.744681,NaN,mua,1482.47,NaN,NaN,False
263,296,None,None,None,None,None,None,None,None,False,...,296,0.081666,2.397622,1.000000,0.000098,mua,285.57,NaN,NaN,True
264,297,"[0.6, 0.7, 0.7, 0.65]","[0.010918277435121127, 0.014782509271753952, 0...","[30, 30, 40, 50]","[surface_LC, surface_LC, surface_LC, surface_LC]","[5, 5, 5, 5]","[4, 4, 4, 4]","[5, 5, 5, 5]","[post, pre, post, post]",True,...,297,0.042272,2.040769,1.000000,0.000446,sua,442.97,"[0.036943320655190535, 0.04178034356952288, 0....","[0.9648659025702908, 0.9512776482486792, 0.944...",True
265,298,"[0.55, 0.5, 0.55, 0.55]","[0.01620521015932106, 0.016074350674171, 0.013...","[30, 30, 40, 50]","[surface_LC, surface_LC, surface_LC, surface_LC]","[5, 5, 5, 5]","[4, 4, 4, 4]","[5, 5, 5, 5]","[post, pre, post, post]",True,...,298,0.028097,3.540009,1.000000,0.000093,noise,455.52,"[0.032577321618421376, 0.042002259768385626, 0...","[0.9618594891829754, 0.9716869200463271, 0.955...",False


In [ ]:
session_list

['behavior_717121_2024-06-15_10-00-58', 'behavior_716325_2024-05-31_10-31-14']

In [ ]:
process('behavior_751766_2025-02-14_11-37-11', data_type='curated')
# process('behavior_717121_2024-06-15_10-00-58')

behavior_751766_2025-02-14_11-37-11 start


/root/capsule/code/beh_ephys_analysis/opto_tagging.py:351: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  unit_qc = unit_qc.apply(pd.to_numeric, errors='ignore')


78 out of 205 units pass quality control
{'samples_to_keep': [-30, 60], 'y_neighbors_to_keep': 3, 'orginal_loc': False}
Merging file 0 out of 205
Merging file 50 out of 205
Merging file 100 out of 205
Merging file 150 out of 205
Merging file 200 out of 205
PDF files in '/root/capsule/scratch/751766/behavior_751766_2025-02-14_11-37-11/ephys/opto/curated/figures' merged into '/root/capsule/scratch/751766/behavior_751766_2025-02-14_11-37-11/ephys/opto/curated/behavior_751766_2025-02-14_11-37-11_opto_tagging.pdf' successfully.
14 out of 205 units pass quality control and opto tagging


/root/capsule/code/beh_ephys_analysis/opto_tagging.py:351: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  unit_qc = unit_qc.apply(pd.to_numeric, errors='ignore')


78 out of 205 units pass quality control
{'samples_to_keep': [-30, 60], 'y_neighbors_to_keep': 3, 'orginal_loc': False}
Merging file 0 out of 205
Merging file 50 out of 205
Merging file 100 out of 205
Merging file 150 out of 205
Merging file 200 out of 205
PDF files in '/root/capsule/scratch/751766/behavior_751766_2025-02-14_11-37-11/ephys/opto/curated/figures' merged into '/root/capsule/scratch/751766/behavior_751766_2025-02-14_11-37-11/ephys/opto/curated/behavior_751766_2025-02-14_11-37-11_opto_tagging.pdf' successfully.
18 out of 205 units pass quality control and opto tagging
Random Forest R² Score fast: 0.7925
Feature Importance fast - motion: 0.40, amp: 0.33, motion*fr: 0.27
Random Forest R² Score slow: 0.8171
Feature Importance slow - motion: 0.50, amp: 0.39, motion*fr: 0.11


...

Random Forest R² Score fast: 0.8659
Feature Importance fast - motion: 0.22, amp: 0.33, motion*fr: 0.45
Random Forest R² Score slow: 0.9722
Feature Importance slow - motion: 0.05, amp: 0.18, motion*fr: 0.77


/root/capsule/code/beh_ephys_analysis/drift_analysis.py:348: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.plot(temp_bins, np.abs(spike_counts_fast - spike_counts_slow_pre)/(0.5*(spike_counts_fast+spike_counts_slow_pre)), label='abs(fast-slow)')


Random Forest R² Score fast: 0.8033
Feature Importance fast - motion: 0.21, amp: 0.52, motion*fr: 0.27
Random Forest R² Score slow: 0.8947
Feature Importance slow - motion: 0.16, amp: 0.50, motion*fr: 0.34


/root/capsule/code/beh_ephys_analysis/drift_analysis.py:348: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.plot(temp_bins, np.abs(spike_counts_fast - spike_counts_slow_pre)/(0.5*(spike_counts_fast+spike_counts_slow_pre)), label='abs(fast-slow)')


Merging file 0 out of 9
PDF files in '/root/capsule/scratch/751766/behavior_751766_2025-02-14_11-37-11/ephys/opto/curated/drift' merged into '/root/capsule/scratch/751766/behavior_751766_2025-02-14_11-37-11/ephys/opto/curated/behavior_751766_2025-02-14_11-37-11_drift.pdf' successfully.
behavior_751766_2025-02-14_11-37-11 stop


In [ ]:
ephys_opto_preprocessing('behavior_717121_2024-06-15_10-00-58', 'curated', 'soma')

behavior_717121_2024-06-15_10-00-58


/opt/conda/lib/python3.9/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.7 is recommended
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter return_scaled from spike_amplitudes extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter method_kwargs from unit_locations extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter sparsity from quality_metrics extension
  warnings.warn(f"Removing legacy parameter {p} from

In [ ]:
opto_tagging_df = opto_plotting_session('behavior_717121_2024-06-15_10-00-58', 'curated', 'soma', plot=True, resp_thresh=0.4, lat_thresh=0.02)

/opt/conda/lib/python3.9/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.7 is recommended
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter return_scaled from spike_amplitudes extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter method_kwargs from unit_locations extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter sparsity from quality_metrics extension
  warnings.warn(f"Removing legacy parameter {p} from

In [ ]:
ephys_opto_crosscorr('behavior_717121_2024-06-15_10-00-58', 'curated')
opto_wf_preprocessing('behavior_717121_2024-06-15_10-00-58', 'curated', 'soma', load_sorting_analyzer=False)

/opt/conda/lib/python3.9/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.7 is recommended
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter return_scaled from spike_amplitudes extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter method_kwargs from unit_locations extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter sparsity from quality_metrics extension
  warnings.warn(f"Removing legacy parameter {p} from

compute_waveforms:   0%|          | 0/6396 [00:00<?, ?it/s]

ValueError: Folder already exists /root/capsule/scratch/717121/behavior_717121_2024-06-15_10-00-58/ephys/opto/curated/opto_waveforms.zarr

In [ ]:
pip install ipykernel

In [ ]:
pip install Ipykernel

In [ ]:

Parallel(n_jobs=3)(delayed(process)(session) for session in session_list)

In [ ]:
plt.close('all')
data_type = 'raw'
session = session_list[0]
opto_tagging_df = opto_plotting_session(session, data_type, 'soma', plot=True, resp_thresh=0.4, lat_thresh=0.02)
plt.close('all')
opto_tagged_spike_stability(session, data_type, 'soma', opto_tagging_df=opto_tagging_df)

/root/capsule/code/beh_ephys_analysis/opto_tagging.py:345: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  unit_qc = unit_qc.apply(pd.to_numeric, errors='ignore')


95 out of 271 units pass quality control
{'samples_to_keep': [-30, 60], 'y_neighbors_to_keep': 3, 'orginal_loc': False}


/root/capsule/code/beh_ephys_analysis/opto_tagging.py:386: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  opto_wf = opto_wf.apply(pd.to_numeric, errors='ignore')


Merging file 0 out of 271
Merging file 50 out of 271
Merging file 100 out of 271
Merging file 150 out of 271
Merging file 200 out of 271
Merging file 250 out of 271
PDF files in '/root/capsule/scratch/751004/behavior_751004_2024-12-19_11-50-37/ephys/opto/raw/figures' merged into '/root/capsule/scratch/751004/behavior_751004_2024-12-19_11-50-37/ephys/opto/raw/behavior_751004_2024-12-19_11-50-37_opto_tagging.pdf' successfully.
5 out of 271 units pass quality control and opto tagging


/root/capsule/code/beh_ephys_analysis/opto_tagging.py:458: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  opto_tagging_df_focus = opto_tagging_df_sorted[(opto_tagging_df_sorted['opto_pass'] == True) & (opto_tagging_df['decoder_label'] != 'artifact') & (opto_tagging_df['decoder_label'] != 'noise')].copy()


In [ ]:
session_list = ['']

In [ ]:
for session in session_list:
    ephys_opto_preprocessing(session, 'curated', 'soma')
    opto_wf_preprocessing(session, 'curated', 'soma', load_sorting_analyzer=False)
    opto_plotting_session(session, 'curated', plot=True)
    plt.close('all')

In [ ]:
session = 'behavior_751004_2024-12-20_13-26-11'
data_type = 'raw'
target = 'soma'
target_unit_ids = [16]
# pass_df = opto_plotting_session(session, data_type, target, plot=True,  resp_thresh=0.5, lat_thresh=0.015, target_unit_ids=target_unit_ids)

In [ ]:
data_type = 'raw'
target = 'soma'
for session in session_list:
    combine_qc = opto_plotting_session(session, data_type, target, plot=True,  resp_thresh=0.5, lat_thresh=0.015)

In [ ]:
session_dir = session_dirs(session)
opto_wf_csv = os.path.join(session_dir[f'opto_dir_{data_type}'], f'{session}_opto_waveform_metrics.csv')
opto_wf = pd.read_csv(opto_wf_csv)

In [ ]:
pip3 install --no-cache-dir numpy

SyntaxError: invalid syntax (4263930493.py, line 1)